In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np

# Sandia presentation _ 2016

## Pyomo fundamentals
#### Setting started: the model
We create a local variable to hold the model we are about to contruct. By convention, we write `model = ConcreteModel()`. We can name the model differently, but we will need to tell the Pyomo cript the object name through the command line.

#### Populating the model: variables
You assing an unique name to the variable `model.name_of_variable = Var()`. Then, you can define the variable domain (e.g., `within=Binary`); if missing, the domain is assumed to be `Reals`. 

#### Defining the objective
In the objective funtion, `expr` can be an expression or any function-like object that return an expression. If `sense` is omitted, Pyomo assumes minimization. 

#### Defining the problem: constraints
Constraints are defined as following: `model.name_of_constraint = Constraint(expr = ...)`. We can either create constraint by constraint, or we can create a list of constraints. For creating a list, we will do the following: 
- `model.list_of_constraints = ConstraintList()` 
- `model.list_of_constraints.add(.... <=)`


In [4]:
#Example
model = ConcreteModel()

model.x = Var(initialize = -1.2, bounds =(-2,2))
model.y = Var(within=Integers, bounds =(-2,1))

model.obj = Objective(
    expr= (1-model.x)**2 + 100*(model.y-model.x**2)**2,
    sense=minimize)


#### Higher-dimensional components
- `model.a_vector = Var(IDX)`
- `model.a_matrix = Var(IDX_A, IDX_B)`
While indexed variables look like matrices, they are not.

#### Manipulating indices: list comprehensions
Paralel to $\sum\limits_{i=IDX} b_{i} \leq a$


In [3]:
model = ConcreteModel()
model.IDX = range(10)
model.a = Var()
model.b = Var(model.IDX)
model.c1 = Constraint(
    expr = sum(model.b[i] for i in model.IDX) <= model.a)

#### Generating and managing indices: Sets and RangeSet
Sets are objects for managing multidimensional indices. You should   `Set(initialize = [1,2,5])` the set that you want. The `initialize` option can accpet sets, lists, and tuples. You can alternatively use `RangeSet` if you want a set of sequential integers. 

Pyomo will attempt to infer the "dimensionality" of Set components (that is, the number of apparent indices). 

In [23]:
model = ConcreteModel()
model.IDX = Set(initialize = range(3), dimen=1) #is ZERO based [0,1,2,3,4]
model.IDXS = RangeSet(5) #is ONE based [1,2,3,4,5]

model.IDX2 = model.IDX * model.IDX * model.IDX # I changed the dimensionality to 3

model.IDX2.pprint()

IDX2 : Size=1, Index=None, Ordered=True
    Key  : Dimen : Domain           : Size : Members
    None :     3 : IDX2_index_0*IDX :   27 : {(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 2, 0), (1, 2, 1), (1, 2, 2), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0), (2, 2, 1), (2, 2, 2)}


In [26]:
# creating sparse sets
model = ConcreteModel()
model.IDX = Set( initialize=[1,2,5] )
def lower_tri_filter(model, i, j):
    return j <= i
model.LTRI = Set( initialize = model.IDX * model.IDX,
                 filter = lower_tri_filter )

model.LTRI.pprint()

LTRI : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :    6 : {(1, 1), (2, 1), (2, 2), (5, 1), (5, 2), (5, 5)}


#### Deferred construction: Rules



#### Solving models: the pyomo command
 `help-solvers`
 
### Concrete vs Abstract
#### Concrete models
Data first, then model. 1-pass construction. All data must be present before Python starts processing the model. Easy to script. Very similar to GAMS. 

#### Abstract model
Model first, then data. 2-pass construction. Pyomo stores the basic model declarations, but does not construct the actual objects; the details on how to construct the component hidden in function or `rules`. At creation time, data is applied to the abstract declaration to create a concrete instance. Encourages generic modeling and model reuse.  

In [5]:
DATA = {1: 10, 2: 21, 5:42}

In [9]:
DATA[1]

10

### Example 1
Determine the set of 𝑃 warehouses chosen from 𝑁
candidates that minimizes the total cost of serving all
customers 𝑀 where 𝑑𝑛,𝑚 is the cost of serving customer 𝑚
from warehouse location 𝑛.

### Example 2:  concrete knapsack
